In [2]:
# this file is to generate new schedule (ie. stop_times.txt) when add or remove bus routes

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import folium

**Below is a temporary block for typing in parameters for new route 6A. will convert to read in version.**

1. get stop_id of route 30, 34 and 36, which are combined into the new route "6A"
  
  a. **trips.txt** maps each route_id with multiple trip_id. **stop_times.txt** maps each trip_id to multiple stop_id. 
  
  b. we can get all the stop_id for route 30, 34, 36 from file **trips.txt** and **stop_times.txt**

In [2]:
# get stops for route 30, 34, 36 
# the functions can also be used to get stops for other routes. 

def get_trips_for_routes(routes = [30,34,36]):
    ### GET TRIPS FOR CERTAIN ROUTES ###
    # para: routes in list (float number)
    # return: trip_id mapping with rotue numbers in dictionary
    
    trips_dict = dict() # A dictionary mapping routes with multiple trips
    
    # read trips.txt file, keep only 2 columns of routes_short_name and trip_id
    trips = (pd.read_csv(r"mmt_gtfs/trips.txt", sep = ",", engine = "python")
            [['route_short_name','trip_id']])
    
    # for each routes (30,34,36), select their trips and update into stop_dict
    for r in routes:
        trip_id = trips.loc[trips['route_short_name'] == r,:]
        tp = trip_id["trip_id"].tolist()
        
        trips_dict.update({r:tp})

    return(trips_dict)


def get_stops_for_routes(routes = [30,34,36]):
    ### GET STOPS FOR CERTIAN ROUTES with stop_squence###
    # para: routes in list
    # return: a dictionary {route_id:[[stop_id,stop_sequence]]}
    #         a list contains all stop_id
    # calling function: get_trips_for_routes
    
    tp_dict = get_trips_for_routes(routes) # get trip_id from route_id
    
    # read stop_times.txt, keep only 3 columns 
    stop_times = (pd.read_csv(r"mmt_gtfs/stop_times.txt", sep = ",", engine = "python")
                  [['trip_id','stop_id','stop_sequence']])        

    stop_dict = dict() # a dictionary mapping route numbers with stops
    stop_list = list() # storing all stops no matter which routes it was under
    
    for r, tp in tp_dict.items():
        i = 0
        for t in tp:
            stop_id = (stop_times.loc[stop_times['trip_id']==t,:]
                      [['stop_id','stop_sequence']])

            st = np.array(stop_id).tolist()
            stop_dict.update({r:st}) # update route:[stop_id, stop_sequence] into dictionary
        
            st_l = stop_id['stop_id'].tolist()
            stop_list += st_l
            
            # for quicker running, we just take the first 6 trips of each route
            # (given that the 6th trip of route 30 is different from the first 5)
            i += 1
            if i >= 6:
                break 
    
    stop_list_unique = list(set(stop_list)) # remove duplicate
    
    return stop_dict, stop_list_unique
        
st_dict, st_list = get_stops_for_routes([30,34,36])



In [5]:
st_dict

{30: [[7100, 1],
  [7169, 2],
  [7167, 3],
  [7209, 4],
  [7633, 5],
  [9167, 6],
  [9811, 7],
  [9277, 8],
  [9243, 9],
  [9439, 10],
  [9137, 11],
  [9745, 12],
  [9951, 13],
  [9645, 14],
  [9291, 15],
  [9432, 16],
  [9719, 17],
  [9677, 18],
  [9800, 19]],
 34: [[7100, 1],
  [1886, 2],
  [1206, 3],
  [1555, 4],
  [1873, 5],
  [9203, 6],
  [9693, 7],
  [9111, 8],
  [9405, 9],
  [9171, 10],
  [9101, 11],
  [9725, 12],
  [9787, 13],
  [9237, 14],
  [9175, 15],
  [9373, 16],
  [9918, 17],
  [9364, 18],
  [9100, 19],
  [9642, 20],
  [9378, 21],
  [9564, 22],
  [9802, 23],
  [9288, 24],
  [1420, 25],
  [1751, 26],
  [1309, 27],
  [1105, 28],
  [7100, 29]],
 36: [[9800, 1],
  [9783, 2],
  [9391, 3],
  [9351, 4],
  [9140, 5],
  [9310, 6],
  [9620, 7],
  [9863, 8],
  [9917, 9],
  [9928, 10],
  [9422, 11],
  [9800, 12]]}

In [11]:
st_list

[7169,
 9591,
 9351,
 9863,
 9100,
 9101,
 1420,
 9745,
 1555,
 9364,
 9237,
 9620,
 9111,
 9753,
 9243,
 9629,
 9373,
 1309,
 9378,
 7209,
 9642,
 9388,
 9645,
 9391,
 9137,
 9778,
 9140,
 1206,
 9783,
 9787,
 7100,
 9277,
 9405,
 9918,
 9917,
 9800,
 9288,
 9802,
 9291,
 9928,
 9677,
 9422,
 9167,
 7633,
 1873,
 9811,
 9171,
 1105,
 9175,
 9432,
 1751,
 9564,
 9693,
 1886,
 9951,
 9439,
 9310,
 9203,
 9719,
 9725,
 7167]

In [19]:
# draw a map 
# https://www.zhihu.com/question/33783546/answer/775946401

# read stops.txt, keep only id, longitute and latitute of stops on route 30, 34 and 30
stops = (pd.read_csv(r"mmt_gtfs/stops.txt", sep = ",", engine = "python")
        [['stop_id', 'stop_lon', 'stop_lat']])
stops = stops.loc[stops['stop_id'].isin(st_list)]


# set map boundary
a = stops.mean()
lat = a.stop_lat
lon = a.stop_lon

# Instantiate a feature group for the stops in the dataframe
stops_6A = folium.map.FeatureGroup()

# add pop-up text to each marker on the map
latitudes = list(stops.stop_lat)
longitudes = list(stops.stop_lon)
labels = list(stops.stop_id)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(sixa_map)    
    
# add incidents to map
sixa_map.add_child(stops_6A)

sixa_map